In [4]:
import csv
import sys
import logging
import os
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim import corpora
import pandas as pd
import json
from gensim import corpora, models, similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
def text_sim(user_text, movie_content):
    #user and movie are lists
    texts = movie_content
    dictionary = corpora.Dictionary(texts)
    
    new_doc = user_text
    new_vec = dictionary.doc2bow(user_text)
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    #corpora.MmCorpus.serialize('m_text.mm', corpus)
    #corpus = corpora.MmCorpus('m_text.mm')
    tfidf = models.TfidfModel(corpus)
    doc_bow = new_vec
    corpus_tfidf = tfidf[corpus]
    
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)
    corpus_lsi = lsi[corpus_tfidf] #bow->tfidf->fold-in-lsi
    vec_bow = new_vec
    vec_lsi = lsi[vec_bow]
    
    index = similarities.MatrixSimilarity(lsi[corpus])
    sims = index[vec_lsi]
    
    sim_id = []
    sim_rate = []
    for i, element in enumerate(sims):
        sim_id.append(i)
        sim_rate.append(element)
    similarity = dict(zip(sim_id, sim_rate))
    similarity = sorted(similarity.items(), key=lambda item:item[1], reverse=True)
    
    return similarity, index

In [6]:
with open('movie_data.json', 'r') as f:
    data = json.load(f)
movie_id = data['movie_index']
movie_title = data['movie_title']
movie_content = data['movie_content']

movie_data = pd.DataFrame.from_dict(data)

user_text = []
f = open('user_data.txt', encoding='utf-8')
for i in f:
    i=i.strip().split('\n')
    user_text.extend(i)
f.close()

In [13]:
s, index = text_sim(user_text, movie_content)

2018-09-02 00:39:14,966 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-02 00:39:15,052 : INFO : built Dictionary(9235 unique tokens: ['able', 'academy', 'according', 'action', 'active']...) from 250 documents (total 155554 corpus positions)
2018-09-02 00:39:15,119 : INFO : collecting document frequencies
2018-09-02 00:39:15,119 : INFO : PROGRESS: processing document #0
2018-09-02 00:39:15,133 : INFO : calculating IDF weights for 250 documents and 9234 features (39623 matrix non-zeros)
2018-09-02 00:39:15,162 : INFO : using serial LSI version on this node
2018-09-02 00:39:15,163 : INFO : updating model with new documents
2018-09-02 00:39:15,323 : INFO : preparing a new chunk of documents
2018-09-02 00:39:15,323 : INFO : using 100 extra samples and 2 power iterations
2018-09-02 00:39:15,323 : INFO : 1st phase: constructing (9235, 120) action matrix
2018-09-02 00:39:15,341 : INFO : orthonormalizing (9235, 120) action matrix
2018-09-02 00:39:15,527 : INFO : 2nd phas

In [14]:
#print(movie_data)
rec_list = []
for i in range(0, len(s)):
    index = s[i][0]
    name = movie_data['movie_title'][index]
    rec_list.append(name)

In [15]:
rec_list[:20]

["Singin' in the Rain",
 'Raging Bull',
 'City Lights',
 'The Apartment',
 'Sherlock Jr.',
 'Some_Like_It_Hot',
 'Annie Hall',
 'Goodfellas',
 'Taxi Driver',
 'Young Frankenstein',
 'The Great Dictator',
 'Rocky',
 'Butch Cassidy and the Sundance Kid',
 'The Sting',
 'Once Upon a Time in America',
 'The Usual Suspects',
 'The Godfather',
 'The Godfather: Part II',
 "Schindler's List",
 'Life of Brian']

In [11]:
with open('movie_data.json', 'r') as f:
    data = json.load(f)
movie_title = data['movie_title']
movie_content = data['movie_content']
#250电影数据库with id, title and content words
movie_data = pd.DataFrame.from_dict(data)

In [3]:
import pandas as pd
dfg = pd.DataFrame(dfg)
dfg

,Recommeded movie,Type
0,Raging Bull,"Biography, Drama, Sport"
1,Rocky,"Action, Romance"
2,Taxi Driver,"Crime, Drama"
3,City Lights,"Drama, Thriller"
4,Singin' in the Rain,Documentary
5,Goodfellas,"Crime, Drama"
6,The Apartment,"Drama, Mystery, Romance"
7,Young Frankenstein,Comedy
8,The Night of the Hunter,"Crime, Drama, Film-Noir"
9,Annie Hall,"Comedy, Romance"
